# Function Generation for the Training of λ-Nets

## Specitication of Experiment Settings

In [1]:
#######################################################################################################################################
###################################################### CONFIG FILE ####################################################################
#######################################################################################################################################
sleep_time = 0 #minutes


config = {
    'function_family': {
        'maximum_depth': 5,
        'beta': 1,
        'decision_sparsity': 1,
        'fully_grown': True,
        'dt_type': 'vanilla', #'vanilla', 'SDT'
    },
    'data': {
        'number_of_variables': 6, 
        'num_classes': 2,
        'categorical_indices': [],
        
        'function_generation_type': 'make_classification_trained', # 'make_classification', 'make_classification_trained', 'random_decision_tree', 'random_decision_tree_trained'
        'objective': 'classification', # 'regression'
        
        'x_max': 1,
        'x_min': 0,
        'x_distrib': 'uniform', #'normal', 'uniform',       
                
        'lambda_dataset_size': 5000, #number of samples per function
        'number_of_generated_datasets': 10000,
    }, 
    'computation':{
        'n_jobs': -3,
        'use_gpu': False,
        'gpu_numbers': '0',
        'RANDOM_SEED': 42,   
    }
}


In [2]:
#######################################################################################################################################
########################################### IMPORT GLOBAL VARIABLES FROM CONFIG #######################################################
#######################################################################################################################################
globals().update(config['function_family'])
globals().update(config['data'])
globals().update(config['computation'])

## Imports

In [3]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from itertools import product       # forms cartesian products
from more_itertools import random_product 
from tqdm import tqdm_notebook as tqdm
import pickle

import numpy as np

import pandas as pd
from joblib import Parallel, delayed

import random 
from random import sample 

import os
import sys

from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler

from sympy import Symbol, sympify

        
import seaborn as sns
        
import random 

import warnings

from time import perf_counter



from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from utilities.DecisionTree_BASIC import SDT
from sklearn.metrics import accuracy_score
from sklearn.tree import plot_tree
from matplotlib import pyplot as plt
from IPython.display import Image

import numpy as np
import torch
np.set_printoptions(suppress=True)
torch.set_printoptions(sci_mode=False)

In [4]:
#######################################################################################################################################
###################################################### SET VARIABLES + DESIGN #########################################################
#######################################################################################################################################
os.environ['CUDA_VISIBLE_DEVICES'] = ''

sns.set_style("darkgrid")

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
    
    
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 500)

warnings.filterwarnings('ignore')

In [5]:
from utilities.utility_functions import *
#######################################################################################################################################
####################################################### CONFIG ADJUSTMENTS ############################################################
#######################################################################################################################################

#######################################################################################################################################
################################################## UPDATE VARIABLES ###################################################################
#######################################################################################################################################
globals().update(config['function_family'])
globals().update(config['data'])
globals().update(config['computation'])

#initialize_utility_functions_config_from_curent_notebook(config)

#######################################################################################################################################
###################################################### PATH + FOLDER CREATION #########################################################
#######################################################################################################################################
globals().update(generate_paths(config, path_type='data_creation'))
generate_directory_structure()

#######################################################################################################################################
############################################################ SLEEP TIMER ##############################################################
#######################################################################################################################################
sleep_minutes(sleep_time)

In [6]:
print(path_identifier_function_data)

lNetSize5000_numDatasets10000_var6_class2_make_classification_trained_xMax1_xMin0_xDistuniform_depth5_beta1_decisionSpars1_vanilla_fullyGrown


# Function Generation

In [7]:
if function_generation_type == 'random_decision_tree':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_data_random_decision_tree)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets))
elif function_generation_type == 'random_decision_tree_trained':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_data_random_decision_tree_trained)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets))  
elif function_generation_type == 'make_classification':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_data_make_classification)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets)) 
elif function_generation_type == 'make_classification_trained':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_data_make_classification_decision_tree_trained)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets))    

function_identifier_list = generate_decision_tree_identifier(config)  
identifier_series_list = [pd.Series(result[0],  index=function_identifier_list) for result in result_list]

function_df = pd.DataFrame(data=np.array([result[0] for result in result_list]), columns=function_identifier_list)

X_data_list = [[identifier_series, pd.DataFrame(result[1], columns=['x' + str(i) for i in range(number_of_variables)])] for identifier_series, result in zip(identifier_series_list, result_list)]
y_data_list = [[identifier_series, pd.DataFrame(result[2], columns=['result'])] for identifier_series, result in zip(identifier_series_list, result_list)]
y_data_raw_list = [[identifier_series, pd.DataFrame(result[3], columns=['result_raw'])] for identifier_series, result in zip(identifier_series_list, result_list)]


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:   31.6s
[Parallel(n_jobs=-3)]: Done 426 tasks      | elapsed:   32.6s
[Parallel(n_jobs=-3)]: Done 1322 tasks      | elapsed:   35.2s
[Parallel(n_jobs=-3)]: Done 2474 tasks      | elapsed:   39.0s
[Parallel(n_jobs=-3)]: Done 3882 tasks      | elapsed:   43.3s
[Parallel(n_jobs=-3)]: Done 5546 tasks      | elapsed:   48.2s
[Parallel(n_jobs=-3)]: Done 7466 tasks      | elapsed:   54.1s
[Parallel(n_jobs=-3)]: Done 9642 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-3)]: Done 10000 out of 10000 | elapsed:  1.0min finished


In [8]:
function_df.head()

,feat0,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,feat11,feat12,feat13,feat14,feat15,feat16,feat17,feat18,feat19,feat20,feat21,feat22,feat23,feat24,feat25,feat26,feat27,feat28,feat29,feat30,split0,split1,split2,split3,split4,split5,split6,split7,split8,split9,split10,split11,split12,split13,split14,split15,split16,split17,split18,split19,split20,split21,split22,split23,split24,split25,split26,split27,split28,split29,split30,lp0,lp1,lp2,lp3,lp4,lp5,lp6,lp7,lp8,lp9,lp10,lp11,lp12,lp13,lp14,lp15,lp16,lp17,lp18,lp19,lp20,lp21,lp22,lp23,lp24,lp25,lp26,lp27,lp28,lp29,lp30,lp31
0,0.624,0.606,0.496,0.691,0.334,0.318,0.636,0.326,0.568,0.310,0.638,0.550,0.685,0.497,0.657,0.550,0.507,0.543,-1.000,0.601,0.569,0.578,0.493,-1.000,0.792,0.653,0.460,0.480,0.743,0.620,-1.000,0.000,3.000,3.000,5.000,4.000,3.000,1.000,4.000,3.000,0.000,3.000,4.000,0.000,2.000,1.000,3.000,4.000,3.000,-1.000,2.000,5.000,5.000,1.000,-1.000,0.000,5.000,3.000,4.000,4.000,2.000,-1.000,0.051,0.505,0.731,0.940,0.006,0.385,1.000,1.000,0.030,1.000,0.657,0.959,0.322,0.875,1.000,0.020,0.000,0.000,1.000,0.000,0.414,0.025,0.016,0.152,1.000,0.103,0.919,0.167,0.167,1.000,0.000,0.000
1,0.485,0.496,0.611,0.591,0.678,0.577,0.541,0.482,0.391,0.441,0.654,0.428,0.586,0.656,0.597,0.232,0.612,0.678,0.657,0.571,0.484,0.458,0.803,0.644,0.551,0.475,0.410,0.509,0.468,0.738,0.440,5.000,2.000,3.000,3.000,0.000,2.000,1.000,3.000,1.000,5.000,2.000,0.000,1.000,5.000,0.000,4.000,0.000,3.000,3.000,2.000,1.000,3.000,0.000,5.000,3.000,1.000,0.000,2.000,2.000,5.000,2.000,0.250,0.008,0.147,1.000,0.000,0.667,0.804,0.996,0.920,0.993,0.929,0.663,0.295,0.967,0.986,0.400,0.121,0.859,0.031,0.155,0.984,0.655,0.925,0.012,0.124,0.514,0.733,0.300,0.883,0.430,0.900,0.000
2,0.490,0.614,0.617,0.359,0.434,0.411,0.714,0.326,0.557,0.559,0.416,0.443,0.532,-1.000,0.585,0.231,0.603,0.418,0.593,0.663,0.440,0.422,0.698,0.252,0.537,0.530,0.661,-1.000,-1.000,0.564,-1.000,3.000,5.000,1.000,4.000,4.000,5.000,5.000,3.000,5.000,2.000,1.000,2.000,2.000,-1.000,3.000,4.000,1.000,4.000,1.000,0.000,0.000,2.000,5.000,5.000,0.000,3.000,5.000,-1.000,-1.000,2.000,-1.000,0.278,0.924,0.041,0.623,0.881,0.990,0.600,0.961,0.750,0.099,0.172,0.904,0.000,0.979,0.329,0.091,1.000,0.033,0.974,0.000,0.072,0.011,0.217,0.708,1.000,1.000,1.000,1.000,0.000,1.000,1.000,1.000
3,0.684,0.588,0.803,-1.000,-1.000,0.796,0.186,-1.000,-1.000,-1.000,-1.000,0.783,0.719,0.879,0.820,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,0.787,-1.000,-1.000,-1.000,0.191,-1.000,-1.000,4.000,1.000,0.000,-1.000,-1.000,0.000,2.000,-1.000,-1.000,-1.000,-1.000,0.000,5.000,0.000,4.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,0.000,-1.000,-1.000,-1.000,1.000,-1.000,-1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.000,1.000,0.000,0.000,1.000,1.000,1.000,1.000,0.000,1.000,0.000,0.000,1.000,1.000
4,0.383,0.298,0.545,0.279,0.282,0.479,0.334,0.236,0.221,0.526,0.534,0.401,0.438,0.678,0.489,-1.000,0.237,-1.000,0.559,0.638,0.640,0.218,0.381,0.597,0.390,0.584,0.728,0.631,0.384,0.445,0.327,2.000,1.000,5.000,1.000,5.000,4.000,4.000,1.000,4.000,4.000,1.000,1.000,1.000,5.000,2.000,-1.000,1.000,-1.000,3.000,1.000,4.000,4.000,5.000,2.000,4.000,2.000,1.000,2.000,1.000,1.000,1.000,0.000,0.000,1.000,0.034,0.000,0.000,0.833,0.167,0.075,0.750,0.588,0.965,0.267,0.993,0.487,0.940,0.295,0.979,0.773,0.504,0.897,0.333,0.021,0.683,0.290,0.571,0.364,0.056,0.135,0.933,0.571,0.004


In [9]:
X_data_list[0][0].head(10)

feat0   0.624
feat1   0.606
feat2   0.496
feat3   0.691
feat4   0.334
feat5   0.318
feat6   0.636
feat7   0.326
feat8   0.568
feat9   0.310
dtype: float64

In [10]:
X_data_list[0][1].head(10)

,x0,x1,x2,x3,x4,x5
0,0.492,0.566,0.458,0.524,0.386,0.493
1,0.528,0.658,0.356,0.272,0.549,0.202
2,0.577,0.599,0.350,0.289,0.576,0.297
3,0.596,0.375,0.397,0.535,0.596,0.770
4,0.538,0.392,0.409,0.606,0.345,0.744
5,0.574,0.546,0.226,0.177,0.634,0.249
6,0.415,0.548,0.552,0.720,0.219,0.643
7,0.640,0.494,0.491,0.516,0.645,0.663
8,0.400,0.531,0.288,0.442,0.082,0.339
9,0.528,0.641,0.587,0.557,0.630,0.527


In [11]:
y_data_list[0][0].head(10)

feat0   0.624
feat1   0.606
feat2   0.496
feat3   0.691
feat4   0.334
feat5   0.318
feat6   0.636
feat7   0.326
feat8   0.568
feat9   0.310
dtype: float64

In [12]:
y_data_list[1][1].head(10)

,result
0,1
1,1
2,1
3,1
4,1
5,0
6,0
7,1
8,0
9,0


In [13]:
y_data_list[2][1].head(10)

,result
0,0
1,1
2,1
3,1
4,0
5,0
6,0
7,0
8,0
9,0


In [14]:
y_data_raw_list[2][1].head(10)

,result_raw
0,0
1,1
2,1
3,1
4,0
5,0
6,0
7,0
8,0
9,0


In [15]:
plot_tree_from_parameters(y_data_list[0][0].values, config)

In [16]:
path_functions = './data/saved_function_lists/functions_' + path_identifier_function_data + '.csv'
function_df.to_csv(path_functions, index=False)

path_X_data = './data/saved_function_lists/X_data_' + path_identifier_function_data + '.pkl'
with open(path_X_data, 'wb') as f:
    pickle.dump(X_data_list, f)
    
path_y_data = './data/saved_function_lists/y_data_' + path_identifier_function_data + '.pkl'
with open(path_y_data, 'wb') as f:
    pickle.dump(y_data_list, f)
